In [1]:

import os
import subprocess
from typing import Optional
from openai import OpenAI
from dotenv import load_dotenv
from abc import ABC, abstractmethod
import json
import re

from prompt import CODE_DEVELOPER_SYSTEMPROMPT, DOCUMENTATION_DEVELOPER_SYSTEMPROMPT, EXECUTE_UIDESIGN_SYSTEMPROMPT
load_dotenv()

client = OpenAI()

class Agent(ABC):
    
    def __init__(self, 
        model: Optional[str] = None, 
        system_prompt: Optional[str] = None
    ) -> None:
        
        model = model if model else os.getenv("OPENAI_DEFAULT_MODEL")
        
        self.memory = []
        self.model = model
        
        if system_prompt:
            self.memory.append({"role": "system", "content": system_prompt})
    
    def add_to_memory(self, role, message):
        self.memory.append({"role": role, "content": message})

    def get_schema(self):
        return self.function
        
    @abstractmethod
    def run(self,prompt):
        """User must define this method. Run the agent"""
    
class ConversationAgent(Agent):
    
    def __init__(self, 
        model: Optional[str] = None, 
        system_prompt: Optional[str] = None,
        name_agent: Optional[str] = None,
        description: Optional[str] = None
    ) -> None:
        super().__init__(model,system_prompt)

        self.name_agent = name_agent
        self.description = description

        self.function={
            "name": self.name_agent,
            "description": self.description,
            "parameters": {
                "type": "object",
                "properties": {
                    "user_prompt": {
                        "type": "string",
                        "description": "User prompt to the agent"
                    }
                },
                "required": ["user_query"]
            }
        }
    
    def run(self,prompt):

        self.add_to_memory("user", prompt)
        
        chat_completion = client.chat.completions.create(
            messages=self.memory,
            model=self.model,
        )
        response=chat_completion.choices[0].message.content
        
        self.add_to_memory("assistant", response)
        
        return response

class CodeExecutionAgent(Agent):
    def __init__(self, 
        model: Optional[str] = None, 
        system_prompt: Optional[str] = None
    ) -> None:
        super().__init__(model,system_prompt)

        self.name_agent = "CodeExecutionAgent"
        self.description = "Agent to execute Python code safely"

        self.function = {
            "name": self.name_agent,
            "description": self.description,
            "parameters": {
                "type": "object",
                "properties": {
                    "code": {
                        "type": "string",
                        "description": "Python code to execute"
                    }
                },
                "required": ["code"]
            }
        }

    def run(self, prompt):
        
        def extract_code_from_string(code_string):
            # Define a regular expression pattern to capture the code between triple backticks
            pattern = r"```python(.*?)```"

            # Use re.DOTALL to ensure that newline characters are matched as well
            matches = re.search(pattern, code_string, re.DOTALL)

            if matches:
                # Extract the code without the backticks
                code = matches.group(1).strip()
                return code
            else:
                return None
        
        code_to_execute = extract_code_from_string(prompt)
        try:
            # Use an isolated environment to execute code, ex. a Docker container
            completed_process = subprocess.run(
                ['python3', '-c', code_to_execute], 
                capture_output=True, 
                text=True,
                timeout=5  # You can set a timeout to prevent long running code
            )
            print(f"completed_process:{completed_process}")
            print(f"completed_process.stdout:{completed_process.stdout}")
            return completed_process.stdout
        except subprocess.SubprocessError as e:
            return str(e)

class ManagerAgent(Agent):
    
    def __init__(self, 
        model: Optional[str] = None, 
        system_prompt: Optional[str] = None,
        list_agents: Optional[list] = None
    ) -> None:
        super().__init__(model,system_prompt)
        
        self.list_agents=list_agents
    
    
    def get_agent_list_schemas(self):
        return [agent.get_schema() for agent in self.list_agents]   
    
    def run(self,prompt):
        
        self.add_to_memory("user", prompt)
        
        prompt_agent_alternatives = [{"role": "user", "content": f"[PROMPT]\n{prompt}\n\n[AGENTS]\n{self.get_agent_list_schemas()}"}]

        #Check what agent i have to execute to resolve the problem
        chat_completion = client.chat.completions.create(
            messages=prompt_agent_alternatives,
            model=self.model,
            functions=self.get_agent_list_schemas()
        )

        # Get the agent to execute
        agent_name=chat_completion.choices[0].message.function_call.name
        agent_prompt=json.loads(chat_completion.choices[0].message.function_call.arguments)['user_prompt']
                
        print(f"Manager: {agent_prompt}")
        self.add_to_memory("user", agent_prompt)
         
        #get the class agent to execute
        class_agent = next((agent for agent in self.list_agents if agent.name_agent == agent_name), None)
        
        #Execute the agent
        response_agent=class_agent.run(agent_prompt)
        
        print(f"{agent_name}: {response_agent}")
        
        self.add_to_memory(agent_name, response_agent)
        
        return response_agent
    
CodeDeveloper=ConversationAgent(
    model="gpt-4-1106-preview",
    name_agent="CodeDeveloper",
    description="Agent to create code in python",
    system_prompt=CODE_DEVELOPER_SYSTEMPROMPT
)
UIDesignDeveloper=ConversationAgent(
    model="gpt-4-1106-preview",
    name_agent="UIDesignDeveloper",
    description="Agent to resolve UI design problems in python",
    system_prompt=EXECUTE_UIDESIGN_SYSTEMPROMPT
)
DocumentationDeveloper=ConversationAgent(
    model="gpt-4-1106-preview",
    name_agent="DocumentationDeveloper",
    description="Agent to document code",
    system_prompt=DOCUMENTATION_DEVELOPER_SYSTEMPROMPT
)

CodeExecuter=CodeExecutionAgent()

# Manager Agents
manager=ManagerAgent(
    model="gpt-4-1106-preview",
    list_agents=[
        CodeDeveloper,
        CodeExecuter,
        UIDesignDeveloper,
        DocumentationDeveloper
    ],
    system_prompt="You need to choose an agent to resolve your development problem. if you need extra files, you can download it from internet."
)

# input = input("Enter a problem: ")

# print(manager.run("Create a code for tetris game"))

In [2]:
manager.run("Create a code for a person detector using my camera, show me it in display and it must run by 1min and end.")

Manager: Create a Python code that uses a camera to detect a person, display the detection in real-time, and automatically stop the detection after one minute.
CodeDeveloper: ```python
import cv2
import time
import threading

# Load the pre-trained model for frontal face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize the camera
camera = cv2.VideoCapture(0)

# Set a time limit in seconds for the detection
detection_time_limit = 60

# Record the start time
start_time = time.time()

def stop_detection():
    global camera
    time.sleep(detection_time_limit)  # Wait for the specified time limit
    if camera.isOpened():
        camera.release()  # Release the camera resource
    cv2.destroyAllWindows()  # Close all OpenCV windows

# Start a thread that will run the stop_detection function
timing_thread = threading.Thread(target=stop_detection)
timing_thread.start()

while(True):
    # Stop the loop if the camera 

"```python\nimport cv2\nimport time\nimport threading\n\n# Load the pre-trained model for frontal face detection\nface_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')\n\n# Initialize the camera\ncamera = cv2.VideoCapture(0)\n\n# Set a time limit in seconds for the detection\ndetection_time_limit = 60\n\n# Record the start time\nstart_time = time.time()\n\ndef stop_detection():\n    global camera\n    time.sleep(detection_time_limit)  # Wait for the specified time limit\n    if camera.isOpened():\n        camera.release()  # Release the camera resource\n    cv2.destroyAllWindows()  # Close all OpenCV windows\n\n# Start a thread that will run the stop_detection function\ntiming_thread = threading.Thread(target=stop_detection)\ntiming_thread.start()\n\nwhile(True):\n    # Stop the loop if the camera is not available anymore\n    if not camera.isOpened():\n        break\n\n    ret, frame = camera.read()\n    \n    if ret:\n        gray = cv2.c

In [4]:
manager.run("Execute the last code")

KeyError: 'user_prompt'